**GETTING THE USER DETAILS AND SAVING THE DATA TO A CSV FILE** 

In [59]:
!pip install twint

In [63]:
import twint 
from datetime import date

today1 = str(date.today())
print("Today's date:", today1)

first_day=str(date.today().replace(day=1))
print(first_day)
name_of_user=input("HI THERE! PLEASE ENTER YOUR TWITTER USERNAME\n")
c=twint.Config()
c.Username = "{}".format(name_of_user)
c.Store_csv=True
c.Output="data_user.csv"
c.Since=first_day
c.Lang='en'
c.Until=today1
twint.run.Search(c)


Today's date: 2020-03-31
2020-03-01
HI THERE! PLEASE ENTER YOUR TWITTER USERNAME
realDonaldTrump
1244728753617620994 2020-03-30 20:50:35 UTC <realDonaldTrump> White House news conference at 5:00 P.M. Eastern. Thank you!
1244673740866191362 2020-03-30 17:11:59 UTC <realDonaldTrump> On #NationalDoctorsDay, we recognize the remarkable men & women who treat their fellow Americans, find cures for the diseases & illnesses we face, and never waver in their efforts to treat every patient with the dignity, respect, and empathy they deserve. https://www.whitehouse.gov/presidential-actions/proclamation-national-doctors-day-2020/ …
1244584449309892609 2020-03-30 11:17:10 UTC <realDonaldTrump> Nancy Pelosi and the Democrats delayed the Workers Help/Stimulus Bill by over a week, trying to add real “junk” into the Bill. Got some bad things, having nothing to do with those affected by the Virus, included. Republicans need their votes until we WIN BACK THE HOUSE IN 2020!
1244581994614398981 2020-03-30 

In [0]:
import pandas as pd 
import warnings
import re 
import nltk 
import string

warnings.filterwarnings('ignore')

In [65]:
data=pd.read_csv("data_user.csv")
data.shape

(385, 34)

In [0]:
# from datetime import date
# today_main=date.today()
# data['date'] = pd.to_datetime(data['date'])
# data=data[(data.date.dt.month==today_main.month)]

In [67]:
data.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1244728753617620994,1244728753617620994,1585601435000,2020-03-30,20:50:35,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,White House news conference at 5:00 P.M. Easte...,[],[],[],16546,14207,104405,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN
1,1244673740866191362,1244673740866191362,1585588319000,2020-03-30,17:11:59,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,"On #NationalDoctorsDay, we recognize the remar...",[],['https://www.whitehouse.gov/presidential-acti...,[],10692,18925,77597,['#nationaldoctorsday'],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN
2,1244584449309892609,1244584449309892609,1585567030000,2020-03-30,11:17:10,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,Nancy Pelosi and the Democrats delayed the Wor...,[],[],[],36862,42417,185215,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN
3,1244581994614398981,1244581994614398981,1585566445000,2020-03-30,11:07:25,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,Thank you Dr. Hahn & the FDA for your fast app...,['mikedewine'],['https://twitter.com/stevefda/status/12444720...,[],2150,11504,49169,[],[],https://twitter.com/realDonaldTrump/status/124...,False,https://twitter.com/SteveFDA/status/1244472087...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN
4,1244493463355342848,1244493463355342848,1585545337000,2020-03-30,05:15:37,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,Will be interviewed on @foxandfriends at 7:55 ...,['foxandfriends'],[],[],15260,15367,92824,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN


In [68]:
data.shape

(385, 34)

In [69]:
from nltk.corpus import sentiwordnet as swn
from nltk import pos_tag, map_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('sentiwordnet')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**FINDING THE SENTIMENT OF EACH TWEET**

In [0]:
lem = WordNetLemmatizer()
pstem = PorterStemmer()

def pos_senti(data):#takes
    li_swn=[]
    li_swn_pos=[]
    li_swn_neg=[]
    missing_words=[]
    for i in range(len(data.index)):
        text = data.loc[i]['tweet']
        tokens = nltk.word_tokenize(text)
        tagged_sent = pos_tag(tokens)
        store_it = [(word, map_tag('en-ptb', 'universal', tag)) for word, tag in tagged_sent]
        #print("Tagged Parts of Speech:",store_it)

        pos_total=0
        neg_total=0
        for word,tag in store_it:
            if(tag=='NOUN'):
                tag='n'
            elif(tag=='VERB'):
                tag='v'
            elif(tag=='ADJ'):
                tag='a'
            elif(tag=='ADV'):
                tag = 'r'
            else:
                tag='nothing'

            if(tag!='nothing'):
                concat = word+'.'+tag+'.01'
                try:
                    this_word_pos=swn.senti_synset(concat).pos_score()
                    this_word_neg=swn.senti_synset(concat).neg_score()
                    #print(word,tag,':',this_word_pos,this_word_neg)
                except Exception as e:
                    wor = lem.lemmatize(word)
                    concat = wor+'.'+tag+'.01'
                    # Checking if there's a possiblity of lemmatized word be accepted into SWN corpus
                    try:
                        this_word_pos=swn.senti_synset(concat).pos_score()
                        this_word_neg=swn.senti_synset(concat).neg_score()
                    except Exception as e:
                        wor = pstem.stem(word)
                        concat = wor+'.'+tag+'.01'
                        # Checking if there's a possiblity of lemmatized word be accepted
                        try:
                            this_word_pos=swn.senti_synset(concat).pos_score()
                            this_word_neg=swn.senti_synset(concat).neg_score()
                        except:
                            missing_words.append(word)
                            continue
                pos_total+=this_word_pos
                neg_total+=this_word_neg
        li_swn_pos.append(pos_total)
        li_swn_neg.append(neg_total)

        if(pos_total!=0 or neg_total!=0):
            if(pos_total>neg_total):
                li_swn.append(1)
            else:
                li_swn.append(-1)
        else:
            li_swn.append(0)
            
    data['pos_score']=li_swn_pos
    data['neg_score']=li_swn_neg
    data['sentiment_score']=li_swn
    return data


data=pos_senti(data)

In [0]:
data1=data.copy()

**CHECK FOR RETWEETS AND IGNORES THE ONES WHICH ARE RETWEETS**

In [0]:
data.tweet.str.contains('@RT').unique()
data=data[data.retweet==False]

In [12]:
data.tweet=data.tweet.str.lower()
data['tweet']

0      white house news conference at 5:00 p.m. easte...
1      on #nationaldoctorsday, we recognize the remar...
2      nancy pelosi and the democrats delayed the wor...
3      thank you dr. hahn & the fda for your fast app...
4      will be interviewed on @foxandfriends at 7:55 ...
                             ...                        
380    i would find it hard to believe that failed pr...
381    tom steyer who, other than mini mike bloomberg...
382                 congratulations to sleepy joe biden!
383    democrats are working hard to destroy the name...
384    sleepy joe biden’s victory in the south caroli...
Name: tweet, Length: 385, dtype: object

**DROPPING THE DUPLICATE TWEETS**

In [0]:
data=data.drop_duplicates(subset='tweet',keep='first')

In [14]:
data.shape

(385, 37)

**analyze tweets by two steps. First, preprocessing tweets by Ignoring a duplicate words or tweets, medical or educational words, numbers, URLs, mentions, images, punctuation and spaces and removes tweets that contain advertisements, and retweet tweets and filters the dataset by dictionary (Sentiwordnet) and calculating the sentiment of each tweet.**
Second, classifier tweet by niave bayes algorithm and show results monthly, weekly and daily percent**

**REMOVING WORDS RELATED TO EDUCATION** 

I have taken the words from https://www.mondovo.com/keywords/education-keywords

In [0]:
education_dataframe=pd.read_csv("education_keywords.csv")
education=list(education_dataframe['keywords'])


In [0]:
def education_removal(text): 
    inter = list(text.split(" "))
    for i in inter:
        if i in education:
            inter.remove(i)
    text=' '.join(inter)
    return text

data['tweet'] = data['tweet'].apply(lambda x: education_removal(x))

**REMOVING WORDS RELATED TO MEDICAL** 

I have taken the words from https://figshare.com/articles/List_of_Health_Keywords/1084358/1

In [0]:
medical_dataframe=pd.read_csv("medical_keywords.csv")
medical_dataframe

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

medical_dataframe['keywords'] = medical_dataframe['keywords'].apply(lambda x: remove_punct(x))

medical=list(medical_dataframe['keywords'])
medical.remove("depression")

In [0]:
def medical_removal(text): 
    inter = list(text.split(" "))
    for i in inter:
        if i in medical:
            inter.remove(i)
    text=' '.join(inter)
    return text
    
    

data['tweet'] = data['tweet'].apply(lambda x: medical_removal(x))

**REMOVING URLS**

In [19]:
def remove_url(text):

    text = re.sub(r"http\S+", "", text)
    text= re.sub(r"https\S+", "", text)
    return text


data['tweet'] = data['tweet'].apply(lambda x: remove_url(x))

data['tweet'][3]

'thank you dr. hahn & the fda for your fast approval of this respected ohio company recommended by governor @mikedewine. great potential! \xa0…'

**REMOVING PHOTOS** 

In [20]:
def remove_photos(text):

    text = re.sub(r"pic.twitter\S+", "", text)
    return text


data['tweet'] = data['tweet'].apply(lambda x: remove_photos(x))

data['tweet'][10]

'...the cbs news poll said 13 percent of republicans trusted the news media for information about the virus.” michael m. grynbaum @nytimes'

**REMOVING MENTIONS** 

In [0]:
def remove_mentions(text):
    inter = list(text.split(" "))
    for i in inter: 
        if i.startswith("@"):
            inter.remove(i)
    text=' '.join(inter)
    return text

data['tweet'] = data['tweet'].apply(lambda x: remove_mentions(x))

**REMOVING ADVERTISEMENTS**

In [22]:
data=data.drop(data[data.tweet.str.contains("#ad")].index)
data=data.drop(data[data.tweet.str.contains("#sponsored")].index)

data.shape

(385, 37)

**REMOVING HASHTAGS WORDS**

In [0]:
def remove_hashtag(text):
    inter = list(text.split(" "))
    for i in inter: 
        if i.startswith("#"):
            inter.remove(i)
    text=' '.join(inter)
    return text

data['tweet'] = data['tweet'].apply(lambda x: remove_hashtag(x))

**REMOVING NUMBERS, PUNCTIONS, TOKENISATION, STOPWORDS, SYMBOLS** 

Remove punctuations

Tokenization - Converting a sentence into list of words

Remove stopwords

In [0]:
import string
import re
import nltk

**REMOVING PUNCTUATIONS & SYMBOLS & NUMBERS** 

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

puncuation and numbers

In [0]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

data['tweet'] = data['tweet'].apply(lambda x: remove_punct(x))

**REMOVING TOKENIZATION**

In [28]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

data['tweet'] = data['tweet'].apply(lambda x: tokenization(x.lower()))
data.head()


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,pos_score,neg_score,sentiment_score
0,1244728753617620994,1244728753617620994,1585601435000,2020-03-30,20:50:35,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,"[white, house, news, conference, at, pm, easte...",[],[],[],16484,14152,103847,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.00,0.000,0
1,1244673740866191362,1244673740866191362,1585588319000,2020-03-30,17:11:59,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,"[on, we, recognize, the, remarkable, men, wome...",[],['https://www.whitehouse.gov/presidential-acti...,[],10636,18822,77062,['#nationaldoctorsday'],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,2.25,1.750,1
2,1244584449309892609,1244584449309892609,1585567030000,2020-03-30,11:17:10,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,"[nancy, pelosi, and, the, democrats, delayed, ...",[],[],[],36794,42337,184864,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,1.25,1.500,-1
3,1244581994614398981,1244581994614398981,1585566445000,2020-03-30,11:07:25,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,"[thank, you, dr, hahn, the, fda, for, your, fa...",['mikedewine'],['https://twitter.com/stevefda/status/12444720...,[],2145,11491,49055,[],[],https://twitter.com/realDonaldTrump/status/124...,False,https://twitter.com/SteveFDA/status/1244472087...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.50,0.375,1
4,1244493463355342848,1244493463355342848,1585545337000,2020-03-30,05:15:37,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,"[will, be, interviewed, on, at, am, usa, strong]",['foxandfriends'],[],[],15259,15353,92703,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.25,0.125,1


In [29]:
data['tweet']

0      [white, house, news, conference, at, pm, easte...
1      [on, we, recognize, the, remarkable, men, wome...
2      [nancy, pelosi, and, the, democrats, delayed, ...
3      [thank, you, dr, hahn, the, fda, for, your, fa...
4       [will, be, interviewed, on, at, am, usa, strong]
                             ...                        
380    [i, would, find, it, hard, to, believe, that, ...
381    [tom, steyer, who, other, than, mini, mike, bl...
382            [congratulations, to, sleepy, joe, biden]
383    [democrats, are, working, hard, to, destroy, t...
384    [sleepy, joe, biden, s, victory, in, the, sout...
Name: tweet, Length: 385, dtype: object

**REMOVING STOP WORDS**

In [0]:
stopword = nltk.corpus.stopwords.words('english')

In [31]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
data['tweet'] = data['tweet'].apply(lambda x: remove_stopwords(x))
data['tweet']

0      [white, house, news, conference, pm, eastern, ...
1      [recognize, remarkable, men, women, treat, fel...
2      [nancy, pelosi, democrats, delayed, workers, h...
3      [thank, dr, hahn, fda, fast, approval, respect...
4                             [interviewed, usa, strong]
                             ...                        
380    [would, find, hard, believe, failed, president...
381    [tom, steyer, mini, mike, bloomberg, spent, do...
382                [congratulations, sleepy, joe, biden]
383    [democrats, working, hard, destroy, name, repu...
384    [sleepy, joe, biden, victory, south, carolina,...
Name: tweet, Length: 385, dtype: object

**DOING STEMMING**

In [32]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

data['tweet'] = data['tweet'].apply(lambda x: stemming(x))
data['tweet']

0        [white, hous, news, confer, pm, eastern, thank]
1      [recogn, remark, men, women, treat, fellow, am...
2      [nanci, pelosi, democrat, delay, worker, helps...
3      [thank, dr, hahn, fda, fast, approv, respect, ...
4                               [interview, usa, strong]
                             ...                        
380    [would, find, hard, believ, fail, presidenti, ...
381    [tom, steyer, mini, mike, bloomberg, spent, do...
382                      [congratul, sleepi, joe, biden]
383    [democrat, work, hard, destroy, name, reput, c...
384    [sleepi, joe, biden, victori, south, carolina,...
Name: tweet, Length: 385, dtype: object

**DOING LEMMATIZATION** 

In [33]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

data['tweet'] = data['tweet'].apply(lambda x: lemmatizer(x))
data['tweet']

0        [white, hous, news, confer, pm, eastern, thank]
1      [recogn, remark, men, woman, treat, fellow, am...
2      [nanci, pelosi, democrat, delay, worker, helps...
3      [thank, dr, hahn, fda, fast, approv, respect, ...
4                               [interview, usa, strong]
                             ...                        
380    [would, find, hard, believ, fail, presidenti, ...
381    [tom, steyer, mini, mike, bloomberg, spent, do...
382                      [congratul, sleepi, joe, biden]
383    [democrat, work, hard, destroy, name, reput, c...
384    [sleepi, joe, biden, victori, south, carolina,...
Name: tweet, Length: 385, dtype: object

In [35]:
data[['tweet']]

,tweet
0,"[white, hous, news, confer, pm, eastern, thank]"
1,"[recogn, remark, men, woman, treat, fellow, am..."
2,"[nanci, pelosi, democrat, delay, worker, helps..."
3,"[thank, dr, hahn, fda, fast, approv, respect, ..."
4,"[interview, usa, strong]"
...,...
380,"[would, find, hard, believ, fail, presidenti, ..."
381,"[tom, steyer, mini, mike, bloomberg, spent, do..."
382,"[congratul, sleepi, joe, biden]"
383,"[democrat, work, hard, destroy, name, reput, c..."


**JOINING THE WORDS TO CREATE A SENTENCE BACK AGAIN** 

In [36]:
data['tweet']=data['tweet'].str.join(" ")
data['tweet']

0                white hous news confer pm eastern thank
1      recogn remark men woman treat fellow american ...
2      nanci pelosi democrat delay worker helpstimulu...
3      thank dr hahn fda fast approv respect ohio com...
4                                   interview usa strong
                             ...                        
380    would find hard believ fail presidenti candid ...
381    tom steyer mini mike bloomberg spent dollar no...
382                           congratul sleepi joe biden
383    democrat work hard destroy name reput crazi be...
384    sleepi joe biden victori south carolina democr...
Name: tweet, Length: 385, dtype: object

**REMOVING DUPLICATE WORDS**

In [0]:
from collections import OrderedDict

In [38]:
data['tweet'] = (data['tweet'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
data


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,pos_score,neg_score,sentiment_score
0,1244728753617620994,1244728753617620994,1585601435000,2020-03-30,20:50:35,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,white hous news confer pm eastern thank,[],[],[],16484,14152,103847,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.000,0.000,0
1,1244673740866191362,1244673740866191362,1585588319000,2020-03-30,17:11:59,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,recogn remark men woman treat fellow american ...,[],['https://www.whitehouse.gov/presidential-acti...,[],10636,18822,77062,['#nationaldoctorsday'],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,2.250,1.750,1
2,1244584449309892609,1244584449309892609,1585567030000,2020-03-30,11:17:10,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,nanci pelosi democrat delay worker helpstimulu...,[],[],[],36794,42337,184864,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,1.250,1.500,-1
3,1244581994614398981,1244581994614398981,1585566445000,2020-03-30,11:07:25,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,thank dr hahn fda fast approv respect ohio com...,['mikedewine'],['https://twitter.com/stevefda/status/12444720...,[],2145,11491,49055,[],[],https://twitter.com/realDonaldTrump/status/124...,False,https://twitter.com/SteveFDA/status/1244472087...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.500,0.375,1
4,1244493463355342848,1244493463355342848,1585545337000,2020-03-30,05:15:37,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,interview usa strong,['foxandfriends'],[],[],15259,15353,92703,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.250,0.125,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,1233990259802767360,1233990259802767360,1583041179000,2020-03-01,05:39:39,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,would find hard believ fail presidenti candid ...,[],[],[],15157,18587,91078,[],[],https://twitter.com/realDonaldTrump/status/123...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,1.250,1.625,-1
381,1233987865350148097,1233987865350148097,1583040608000,2020-03-01,05:30:08,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,tom steyer mini mike bloomberg spent dollar no...,[],[],[],7594,16391,79552,[],[],https://twitter.com/realDonaldTrump/status/123...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,2.194,1.931,1
382,1233977649552384005,1233977649552384005,1583038172000,2020-03-01,04:49:32,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,congratul sleepi joe biden,[],[],[],8542,11370,79091,[],[],https://twitter.com/realDonaldTrump/status/123...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,1.000,0.000,1
383,1233976658572566529,1233976658572566529,1583037936000,2020-03-01,04:45:36,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,democrat work hard destroy name reput crazi be...,[],[],[],10551,16561,90641,[],[],https://twitter.com/realDonaldTrump/status/123...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.625,0.500,1


**MAKING NEW DATAFRAME FOR OPERATION** 

In [0]:
new_data=data[['tweet','likes_count','replies_count','pos_score','neg_score','sentiment_score']]

In [40]:
new_data

,tweet,likes_count,replies_count,pos_score,neg_score,sentiment_score
0,white hous news confer pm eastern thank,103847,16484,0.000,0.000,0
1,recogn remark men woman treat fellow american ...,77062,10636,2.250,1.750,1
2,nanci pelosi democrat delay worker helpstimulu...,184864,36794,1.250,1.500,-1
3,thank dr hahn fda fast approv respect ohio com...,49055,2145,0.500,0.375,1
4,interview usa strong,92703,15259,0.250,0.125,1
...,...,...,...,...,...,...
380,would find hard believ fail presidenti candid ...,91078,15157,1.250,1.625,-1
381,tom steyer mini mike bloomberg spent dollar no...,79552,7594,2.194,1.931,1
382,congratul sleepi joe biden,79091,8542,1.000,0.000,1
383,democrat work hard destroy name reput crazi be...,90641,10551,0.625,0.500,1


**DOING PROCESS OF VECTORIZATION AND APPLYING THE ALGORITHM** 

In [0]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=1000, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
new_data['tweet'] = vectorizer.fit_transform(new_data['tweet']).toarray()

In [0]:
new_data['tweet'].unique()

labels=new_data['sentiment_score']
value_to_pass=new_data.drop(columns=['sentiment_score'])

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(value_to_pass, labels, test_size=0.4, random_state=0)


In [0]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets
model.fit(value_to_pass,labels)

predicted=list(model.predict(X_test))

**DETERMINING IF THE PERSON IS DEPRESSED OR NOT**

In [45]:
if predicted.count(1)>predicted.count(-1):
    print('The person is NOT DEPRESSED')
elif predicted.count(1)==predicted.count(-1):
    print("The person has a NEUTRAL sentiment")
elif predicted.count(1)<predicted.count(-1):
    print('The person is DEPRESSED')

The person is NOT DEPRESSED


**SHOWCASING THE MONTHLY, WEEKLY, DAILY TWEETS** 

In [46]:
data 

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,pos_score,neg_score,sentiment_score
0,1244728753617620994,1244728753617620994,1585601435000,2020-03-30,20:50:35,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,white hous news confer pm eastern thank,[],[],[],16484,14152,103847,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.000,0.000,0
1,1244673740866191362,1244673740866191362,1585588319000,2020-03-30,17:11:59,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,recogn remark men woman treat fellow american ...,[],['https://www.whitehouse.gov/presidential-acti...,[],10636,18822,77062,['#nationaldoctorsday'],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,2.250,1.750,1
2,1244584449309892609,1244584449309892609,1585567030000,2020-03-30,11:17:10,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,nanci pelosi democrat delay worker helpstimulu...,[],[],[],36794,42337,184864,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,1.250,1.500,-1
3,1244581994614398981,1244581994614398981,1585566445000,2020-03-30,11:07:25,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,thank dr hahn fda fast approv respect ohio com...,['mikedewine'],['https://twitter.com/stevefda/status/12444720...,[],2145,11491,49055,[],[],https://twitter.com/realDonaldTrump/status/124...,False,https://twitter.com/SteveFDA/status/1244472087...,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.500,0.375,1
4,1244493463355342848,1244493463355342848,1585545337000,2020-03-30,05:15:37,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,interview usa strong,['foxandfriends'],[],[],15259,15353,92703,[],[],https://twitter.com/realDonaldTrump/status/124...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.250,0.125,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,1233990259802767360,1233990259802767360,1583041179000,2020-03-01,05:39:39,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,would find hard believ fail presidenti candid ...,[],[],[],15157,18587,91078,[],[],https://twitter.com/realDonaldTrump/status/123...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,1.250,1.625,-1
381,1233987865350148097,1233987865350148097,1583040608000,2020-03-01,05:30:08,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,tom steyer mini mike bloomberg spent dollar no...,[],[],[],7594,16391,79552,[],[],https://twitter.com/realDonaldTrump/status/123...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,2.194,1.931,1
382,1233977649552384005,1233977649552384005,1583038172000,2020-03-01,04:49:32,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,congratul sleepi joe biden,[],[],[],8542,11370,79091,[],[],https://twitter.com/realDonaldTrump/status/123...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,1.000,0.000,1
383,1233976658572566529,1233976658572566529,1583037936000,2020-03-01,04:45:36,UTC,25073877,realdonaldtrump,Donald J. Trump,NaN,democrat work hard destroy name reput crazi be...,[],[],[],10551,16561,90641,[],[],https://twitter.com/realDonaldTrump/status/123...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '25073877', 'username': 'realDona...",NaN,NaN,NaN,NaN,0.625,0.500,1


**MAKING A NEW DATAFRAME WITH DATE AND TIME VALUES**

In [0]:
calender=data[['tweet','likes_count','replies_count','pos_score','neg_score','sentiment_score','date','time']]

In [48]:
# calender['date']=calender['date'].astype("datetime64[ns]")
calender.shape

(385, 8)

In [49]:
calender.date.dtype

dtype('O')

**FINDING OUT TODAY'S DAY**

In [50]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2020-03-31


In [0]:
save=list(calender['date'])


**GETTING THE NUMBER OF DAYS BEFORE A TWEET WAS TWEETED**

**OBTAINING DAILY TWEETS** 

In [84]:
data1['date'] = pd.to_datetime(data1['date'])
daily=data1[(data1.date.dt.month==today.month) & (data1.date.dt.day==today.day) & (data1.sentiment_score==-1)]
daily_tweet=list(daily['tweet'])

daily_json=json.dumps(daily_tweet)
print(daily_json)

with open("dail.json", "w") as write_file:
    json.dump(daily_json, write_file)

[]


**OBTAIING WEEKLY TWEETS**

In [80]:
import datetime
import json
weekno=datetime.datetime.utcnow().isocalendar()[1]
# print(weekno)
data1['date'] = pd.to_datetime(data1['date'])
weekly=data1[(data1.date.dt.week==weekno) & (data1.sentiment_score==-1)]
weekly_tweet=list(weekly['tweet'])

weekly_json=json.dumps(weekly_tweet)
print(weekly_json)

with open("week.json", "w") as write_file:
    json.dump(weekly_json, write_file)

14
["Nancy Pelosi and the Democrats delayed the Workers Help/Stimulus Bill by over a week, trying to add real \u201cjunk\u201d into the Bill. Got some bad things, having nothing to do with those affected by the Virus, included. Republicans need their votes until we WIN BACK THE HOUSE IN 2020!"]


**OBTAINING MONTHLY TWEETS** 

In [83]:
data1['date'] = pd.to_datetime(data1['date'])
monthly=data1[(data1.date.dt.month==today.month) & (data1.sentiment_score==-1)]


monthly_tweet=list(monthly['tweet'])

monthly_json=json.dumps(monthly_tweet)
print(monthly_json)


with open("month.json", "w") as write_file:
    json.dump(monthly_json, write_file)


["Nancy Pelosi and the Democrats delayed the Workers Help/Stimulus Bill by over a week, trying to add real \u201cjunk\u201d into the Bill. Got some bad things, having nothing to do with those affected by the Virus, included. Republicans need their votes until we WIN BACK THE HOUSE IN 2020!", "I am a great friend and admirer of the Queen & the United Kingdom. It was reported that Harry and Meghan, who left the Kingdom, would reside permanently in Canada. Now they have left Canada for the U.S. however, the U.S. will not pay for their security protection. They must pay!", "...alone attracted 6.2 million viewers for the president\u2019s briefing \u2014 an astounding number for a 6 p.m. cable broadcast, more akin to the viewership for a popular prime-time sitcom...", "Because the \u201cRatings\u201d of my News Conferences etc. are so high, \u201cBachelor finale, Monday Night Football type numbers\u201d according to the @nytimes, the Lamestream Media is going CRAZY. \u201cTrump is reaching t

In [55]:
! pip install pyphi

In [0]:
f=open("{}.json".format(name_of_user),"w")

In [0]:
main1={"daily":daily_json,"weekly":weekly_json ,"monthly":monthly_json}

In [0]:
from pyphi import jsonify

jsonify.dump(main1,f)
f.close()